# 0. 事前準備

## 安裝 gensim 套件
pip install gensim

### 相依的其他套件
##### NumPy
##### SciPy
##### Cython
在進行模型訓練的時候，如果沒有安裝Cython，便只能使用單核，執行速度較慢

## 安裝 jieba 套件
pip install jieba

# 1. 資料來源：中文維基百科
https://dumps.wikimedia.org/zhwiki/20170201/

檔名：zhwiki-20170201-pages-articles.xml.bz2

但此處只取前 5000 篇文章作為範例檔案

# 2. 採用 jieba 進行中文斷詞
結巴中文分詞支持的三種分詞模式包括：
### 1. 精確模式：將句子最精確地切開，適合文本分析。
寫法：words = jieba.cut(content, cut_all=False)
### 2.全模式：把句子中所有可以成詞的詞語都掃描出來，速度快。
寫法：words = jieba.cut(content, cut_all=True)
### 3.搜索引勤模式：在精確模式的基礎上對長詞再次切分，提高召回率，適合用於搜尋引擎分詞。
寫法：jieba.cut_for_search(Content)

In [1]:
import io
import logging
import multiprocessing
import jieba
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence, Text8Corpus

In [2]:
def segment(input_path, output_path):
    """
    利用結巴套件斷字，使用預設詞庫（最基本的用法）
    """    
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    
    # jieba custom setting.
    jieba.load_userdict('userdict_modified.txt')

    # load stopwords set
    stopwordset = set()
    with io.open('stop_words.txt', 'r', encoding='utf-8') as sw:
        for line in sw:
            stopwordset.add(line.strip('\n'))

    output = open(output_path,'w')
    texts_num = 0 
    
    with open(input_path,'r') as content :
        for line in content:
            words = jieba.cut(line, cut_all=False)  # 精確模式
            for word in words:
                if word not in stopwordset:
                    output.write(word.encode('utf-8') +' ')                        
            texts_num += 1
            if texts_num % 500 == 0:
                logging.info("已完成前 %d 行的斷詞" % texts_num)
    
    output.close()

In [29]:
segment('datasource/wiki_zh_tw_part.txt', 'datasource/wiki_seg_part.txt')

# 3. 使用 gensim 套件

Gensim, programmed by Radim Řehůřek, is an open source package suitable to analyze large textual collections by the usage of parallel distributable online algorithms.

## 3.1 訓練、儲存模型

In [4]:
def build_word2vec(corpus_path, savePath):

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    
    corpus = LineSentence(corpus_path) # 採用內建函式讀入 corpus
    
    num_features = 250 # 詞向量的維度（即：隱藏層的神經元個數）
    window_size = 5 # 考慮前後文的範圍
    min_count = 5 # 若某字彙出現的次數小於 min_count，那它就不會被視為訓練對象，會被丟棄
    sg_type = 0 # sg=0，CBOW  # sg=1，skip-gram
    
    model = Word2Vec(corpus, size=num_features, sg=sg_type, window=window_size, min_count=min_count, workers=multiprocessing.cpu_count())    
    
    # 儲存模型
    model.save(savePath+'word2vec_model') # gensim 的 model 格式
    model.save_word2vec_format(savePath+'word2vec_model_vec.txt', binary=False) # 原生 C 的 model 格式，以非二進制方式儲存，方便人去查看
    model.save_word2vec_format(savePath+'word2vec_model_vec.bin', binary=True) # 原生 C 的 model 格式，以二進制方式儲存，提高i/o效能
    
    return model

In [28]:
model = build_word2vec('datasource/wiki_seg_part.txt', 'model_result/')

## 3.2 載入模型

In [6]:
#############
## 載入模型 ##
#############

## gensim 的 model 格式
#model_gensim = Word2Vec.load('model_result/word2vec_model')

## 原生 C 的 model 格式
#model_c_txt = Word2Vec.load_word2vec_format('model_result/word2vec_model_vec.txt', binary=False)
#model_c_bin = Word2Vec.load_word2vec_format('model_result/word2vec_model_vec.bin', binary=True)

## 3.3 使用模型

In [7]:
def ps(result):  
    if result:
        for e in result:
            print e[0], e[1]

### (1) model 中的所有詞彙

In [24]:
for key in model.wv.vocab:
    print key

### (2) 特定詞彙的詞向量

In [25]:
# raw NumPy vector of a word
model[u'情人']

### (3) 找出與特定詞彙最相似的其他詞彙

In [10]:
result = model.most_similar(u'情人', topn=5)
ps(result)

2017-02-18 14:52:55,246 : INFO : precomputing L2-norms of word weight vectors


女孩 0.917469084263
媽媽 0.91675645113
續集 0.912169754505
張學友 0.911907672882
客串 0.905912518501


In [11]:
result = model.most_similar(u'北京', topn=5)
ps(result)

上海 0.867828786373
天津 0.815129578114
南京 0.760588943958
重慶 0.756330907345
西安 0.734520614147


In [12]:
result = model.most_similar(u'高雄', topn=5)
ps(result)

新竹 0.861851036549
捷運 0.854273676872
戲院 0.845746219158
高雄市 0.841543316841
嘉義 0.838007032871


In [13]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=5)  # woman+king-man=?
ps(result)

frederick 0.955409049988
tudor 0.949015796185
bach 0.948603689671
philip 0.948344528675
maurice 0.947911918163


In [14]:
result = model.most_similar(positive=[u'女', u'皇帝'], negative=[u'男'], topn=5)   # 女+皇帝-男=?
ps(result)

君主 0.805771589279
統治者 0.783081352711
國王 0.770947754383
教宗 0.763281166553
皇位 0.762493312359


In [15]:
result = model.most_similar(positive=[u'女人', u'皇帝'], negative=[u'男人'], topn=5)   # 女人+皇帝-男人=?
ps(result)

廟號 0.79917371273
君主 0.787366986275
開國 0.785382032394
尊號 0.784314870834
皇位 0.782575488091


### (4) 計算兩個詞彙的相似度

In [16]:
print model.similarity(u"男人", u"女人")

0.961160928823


### (5) 從一堆詞彙裡面找到不匹配的詞彙

In [17]:
print model.doesnt_match(u"早餐 晚餐 午餐 中心".split())

中心


In [18]:
print model.doesnt_match(u"高雄 嘉義 上海".split())

上海


## 3.4 更新模型：訓練新的語料 

In [30]:
# 載入已經訓練好的模型
model_gensim = Word2Vec.load('model_result/word2vec_model')

In [20]:
keyword = u'蝶戀花'

if keyword in model_gensim.vocab:
    result = model_gensim.most_similar(keyword, topn=5)
    ps(result)
else:
    print keyword + u"不在字典中！"

2017-02-18 14:53:40,457 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-02-18 14:53:40,459 : INFO : precomputing L2-norms of word weight vectors


情詩 0.924734652042
豬八戒 0.921241462231
梟雄 0.920531213284
二十首 0.919621646404
離別 0.919123828411


In [31]:
# 新的語料
segment('datasource/news.txt', 'datasource/news_seg.txt')
new_corpus = LineSentence('datasource/news_seg.txt')

# 更新模型：訓練新的語料
model_gensim.train(new_corpus)

# 儲存更新後的模型
model_gensim.save('model_result/word2vec_model_updated')

In [22]:
keyword = u'蝶戀花'
if keyword in model_gensim.wv.vocab:
    result = model_gensim.most_similar(keyword, topn=5)
    ps(result)
else:
    print keyword + u"不在字典中！"

2017-02-18 14:53:57,690 : INFO : precomputing L2-norms of word weight vectors


旅行社 0.887470602989
顧城 0.831960201263
郭德潔 0.818841576576
立群 0.812965035439
保外 0.812554180622


In [23]:
result = model_gensim.most_similar(u'情人', topn=5)
ps(result)

女孩 0.917469084263
媽媽 0.91675645113
續集 0.912169754505
張學友 0.911907672882
客串 0.905912518501
